In [ ]:
# F1 Score achieved  on Kaggle : 1

In [ ]:
import torch
import os
import pandas as pd
from torchvision import transforms, models
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import json

In [ ]:
# Load label map
with open("/content/drive/MyDrive/soil-classification/label_map.json", 'r') as f:
    idx2label = json.load(f)
idx2label = {int(k): v for k, v in idx2label.items()}  # Ensure keys are ints

In [ ]:
# Load model
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(idx2label))
model.load_state_dict(torch.load("/content/drive/MyDrive/soil-classification/resnet18_soil.pth"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()


In [ ]:
# Define test transform
transform_test = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:

# Dataset class
class SoilDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filename']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, row['image_id']


In [ ]:
# Prepare test DataFrame
TEST_DIR = "/content/drive/MyDrive/soil-classification/soil_classification-2025/test/"
test_df = pd.read_csv("/content/drive/MyDrive/soil-classification/soil_classification-2025/test_ids.csv")
test_df['filename'] = test_df['image_id'].apply(lambda x: os.path.join(TEST_DIR, x))

In [ ]:
# Inference
test_dataset = SoilDataset(test_df, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []
image_ids = []

with torch.no_grad():
    for images, ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(1).cpu().numpy()
        predictions.extend(preds)
        image_ids.extend(ids)

In [ ]:
# Save submission
final_df = pd.DataFrame({
    "image_id": image_ids,
    "soil_type": [idx2label[i] for i in predictions]
})
final_df.to_csv("submission.csv", index=False)
print(" submission.csv generated")